<a href="https://colab.research.google.com/github/MS-H2020/Open/blob/main/Open_Images_Object_Detection_RVC_2020_edition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Open Images Object Detection RVC 2020 edition](https://www.kaggle.com/competitions/open-images-object-detection-rvc-2020/code?competitionId=19205&sortBy=voteCount&excludeNonAccessedDatasources=true)

## Refference
1.   [Tensorflow Hub Faster Rcnn](https://www.kaggle.com/code/c7934597/tensorflow-hub-faster-rcnn)



## 0. Setting for Google Colab

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
INPUT_DIR:str = "/content/drive/MyDrive/open-images-object-detection-rvc-2020/01_input"
ANALYSIS_DIR:str = "/content/drive/MyDrive/open-images-object-detection-rvc-2020/03_code-analysis"
MODEL_DIR:str = "/content/drive/MyDrive/open-images-object-detection-rvc-2020//04_model"
SUBMISSION_DIR:str = '/content/drive/MyDrive/open-images-object-detection-rvc-2020/05_submission'
CHECKPOINT_PATH:str = MODEL_DIR

In [4]:
import os, gc, glob, shutil, zipfile
shutil.unpack_archive(INPUT_DIR+'/test.zip', './')

## 1. Importing Modules

In [7]:
import os
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from six import BytesIO
import numpy as np
import xml.etree.ElementTree as et
import ast
import tqdm
from itertools import chain
from xml.dom import minidom
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps
import cv2
import glob
import time

In [5]:
path1 = INPUT_DIR + '/test/'

In [9]:
df = pd.read_csv(INPUT_DIR + '/sample_submission.csv')
df.head()

,ImageId,PredictionString
0,b5d912e06f74e948,/m/05s2s 0.9 0.46 0.08 0.93 0.5 /m/0c9ph5 0.5 ...
1,be137cf6bb0b62d5,/m/05s2s 0.9 0.46 0.08 0.93 0.5 /m/0c9ph5 0.5 ...
2,8d65ca08cb5ce8e8,/m/05s2s 0.9 0.46 0.08 0.93 0.5 /m/0c9ph5 0.5 ...
3,4d3ad1e52ad8c065,/m/05s2s 0.9 0.46 0.08 0.93 0.5 /m/0c9ph5 0.5 ...
4,9b94408691c7d7bf,/m/05s2s 0.9 0.46 0.08 0.93 0.5 /m/0c9ph5 0.5 ...


In [10]:
df.shape

(99999, 2)

In [11]:
ids = []
for i in range(len(df)):
    ids.append(df['ImageId'][i])

In [12]:
ids[0:5]

['b5d912e06f74e948',
 'be137cf6bb0b62d5',
 '8d65ca08cb5ce8e8',
 '4d3ad1e52ad8c065',
 '9b94408691c7d7bf']

In [21]:
img_data=[]
for i in range(len(df)):
    img_data.append(glob.glob('./test/{0}.jpg'.format(ids[i])))

In [22]:
img_data[0:5]

[['./test/b5d912e06f74e948.jpg'],
 ['./test/be137cf6bb0b62d5.jpg'],
 ['./test/8d65ca08cb5ce8e8.jpg'],
 ['./test/4d3ad1e52ad8c065.jpg'],
 ['./test/9b94408691c7d7bf.jpg']]

In [23]:
img_data = list(chain.from_iterable(img_data))

In [24]:
img_data[0:5]

['./test/b5d912e06f74e948.jpg',
 './test/be137cf6bb0b62d5.jpg',
 './test/8d65ca08cb5ce8e8.jpg',
 './test/4d3ad1e52ad8c065.jpg',
 './test/9b94408691c7d7bf.jpg']

In [25]:
def get_prediction_string(result):
    with tf.device('/device:GPU:0'):
        df = pd.DataFrame(columns=['Ymin','Xmin','Ymax', 'Xmax','Score','Label','Class_label','Class_name'])
        min_score=0.01
        for i in range(result['detection_boxes'].shape[0]):
           if (result["detection_scores"][i]) >= min_score:
              df.loc[i]= tuple(result['detection_boxes'][i])+(result["detection_scores"][i],)+(result["detection_class_labels"][i],)+(result["detection_class_names"][i],)+(result["detection_class_entities"][i],)
        return df

In [31]:
import tensorflow.compat.v1
tensorflow.compat.v1.disable_v2_behavior()

In [32]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"
with tf.device('/device:GPU:0'):
    with tf.Graph().as_default():
        detector = tf.Module(module_handle)
        image_string_placeholder = tf.placeholder(tf.string)
        decoded_image = tf.image.decode_jpeg(image_string_placeholder)
        decoded_image_float = tf.image.convert_image_dtype(
            image=decoded_image, dtype=tf.float32)
        module_input = tf.expand_dims(decoded_image_float, 0)
        result = detector(module_input, as_dict=True)
        init_ops = [tf.global_variables_initializer(), tf.tables_initializer()]

        session = tf.Session()
        session.run(init_ops)

ValueError: 'https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1' is not a valid module name. Module names must be valid Python identifiers (e.g. a valid class name).